# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
# Download link 1
# !gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

# !unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Sat Apr 30 10:07:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 37%   39C    P3    41W / 250W |   1461MiB /  8192MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
Numbering = 18
logToWandb = False
if logToWandb: 
    import wandb

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
# !pip install transformers==4.5.0

## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True

# myseed = random.randint(0,1000)
myseed = 0
same_seeds(myseed)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
#     !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [6]:
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

/home/yuting/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnsw

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

In [8]:
train_questions[0]

{'id': 0,
 'paragraph_id': 3884,
 'question_text': '羅馬教皇利奧三世在800年正式加冕誰為羅馬人的皇帝?',
 'answer_text': '查理大帝',
 'answer_start': 141,
 'answer_end': 144}

## Tokenize Data

In [9]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [10]:
answer_start_token = 299
answer_end_token = 300
max_paragraph_len = 180
doc_stride = 70
tokenized_paragraph_len = 300

In [11]:
paragraph_start = max(0, answer_start_token - int(random.uniform(0, max_paragraph_len - (answer_end_token - answer_start_token))/1))
paragraph_end = paragraph_start + max_paragraph_len

paragraph_start, paragraph_end

(148, 328)

In [12]:
mid = int((answer_start_token + answer_end_token) // (2+random.uniform(-1,1)))
paragraph_start = max(0, min(mid - max_paragraph_len // 2, tokenized_paragraph_len - max_paragraph_len))
paragraph_end = paragraph_start + max_paragraph_len

paragraph_start, paragraph_end

(120, 300)

In [13]:
random.randint(0,1)

1

In [14]:
for i in range(0, tokenized_paragraph_len, doc_stride):
    if answer_start_token >= i and answer_end_token <= i+max_paragraph_len:
        paragraph_start = i
        if(random.randint(0,1)==0): break
        
paragraph_end = paragraph_start + max_paragraph_len

paragraph_start, paragraph_end

(140, 320)

In [15]:
int(random.uniform(0, max_paragraph_len - (answer_end_token - answer_start_token)))

46

## Dataset and Dataloader

In [16]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 180
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 70

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # random start
#             paragraph_start = max(0, answer_start_token - int(random.uniform(0, self.max_paragraph_len - (answer_end_token - answer_start_token))))
#             paragraph_start = max(0, answer_start_token - int(random.uniform(0, max_paragraph_len - (answer_end_token - answer_start_token))))
#             mid = (answer_start_token + answer_end_token) // 2
            mid = int((answer_start_token + answer_end_token) // (2+random.uniform(-1,1)))
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))

#             for i in range(0, len(tokenized_paragraph)+1, self.doc_stride):
#                 if answer_start_token >= i and answer_end_token <= i+self.max_paragraph_len:
#                     paragraph_start = i
#                     if(random.randint(0,1)==0): break
                        
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 2

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [17]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob and start_index <= end_index and end_index - start_index <= 20:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
#             answer = answer.replace('「', '').replace('」','')
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

In [18]:
'1233'.replace('2', '')

'133'

## Training

In [19]:
if logToWandb:
    wandb.init(
        # Set the project where this run will be logged
        project="ML2022Spring_HW7",
        name= 'bert{} {}'.format(Numbering,myseed),
    )

In [20]:
num_epoch = 4
validation = True
logging_step = 1000
learning_rate = 1e-5
accum_iter = 4 # batch accumulation parameter
best_acc = 0

optimizer = AdamW(model.parameters(), lr=learning_rate)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps= 5000, num_training_steps=total_steps)
model.train()

print("Start Training ...")

step = 1
for epoch in range(num_epoch): 
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        # https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/02/19/gradient-accumulation.html
        # Gradient Accumulation
#         if (step % accum_iter == 0) or (step % len(train_loader) == 0):
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
            
        step += 1

        ##### TODO: Apply linear learning rate decay #####
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            if logToWandb:
                wandb.log({'Loss/train': train_loss.item() / logging_step, 'step':step})
                wandb.log({'ACC/train': train_acc / logging_step, 'step':step})

            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            if logToWandb:
                wandb.log({'ACC/val': dev_acc / len(dev_loader), 'step':step})
                
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」

model_save_dir = "saved_model{}".format(Numbering) 
print("Saving Model ...", model_save_dir)
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/15845 [00:00<?, ?it/s]

Epoch 1 | Step 1000 | loss = 4.089, acc = 0.125
Epoch 1 | Step 2000 | loss = 1.355, acc = 0.560
Epoch 1 | Step 3000 | loss = 1.049, acc = 0.640
Epoch 1 | Step 4000 | loss = 0.979, acc = 0.659
Epoch 1 | Step 5000 | loss = 0.940, acc = 0.666
Epoch 1 | Step 6000 | loss = 0.885, acc = 0.697
Epoch 1 | Step 7000 | loss = 0.886, acc = 0.693
Epoch 1 | Step 8000 | loss = 0.857, acc = 0.699
Epoch 1 | Step 9000 | loss = 0.760, acc = 0.729
Epoch 1 | Step 10000 | loss = 0.730, acc = 0.739
Epoch 1 | Step 11000 | loss = 0.803, acc = 0.729
Epoch 1 | Step 12000 | loss = 0.774, acc = 0.725
Epoch 1 | Step 13000 | loss = 0.739, acc = 0.723
Epoch 1 | Step 14000 | loss = 0.729, acc = 0.742
Epoch 1 | Step 15000 | loss = 0.697, acc = 0.743
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.786


...

Validation | Epoch 3 | acc = 0.786


  0%|          | 0/15845 [00:00<?, ?it/s]

Epoch 4 | Step 48000 | loss = 0.110, acc = 0.420
Epoch 4 | Step 49000 | loss = 0.226, acc = 0.913
Epoch 4 | Step 50000 | loss = 0.195, acc = 0.927
Epoch 4 | Step 51000 | loss = 0.234, acc = 0.911
Epoch 4 | Step 52000 | loss = 0.210, acc = 0.917
Epoch 4 | Step 53000 | loss = 0.194, acc = 0.918
Epoch 4 | Step 54000 | loss = 0.187, acc = 0.929
Epoch 4 | Step 55000 | loss = 0.230, acc = 0.911
Epoch 4 | Step 56000 | loss = 0.245, acc = 0.916
Epoch 4 | Step 57000 | loss = 0.236, acc = 0.913
Epoch 4 | Step 58000 | loss = 0.221, acc = 0.925
Epoch 4 | Step 59000 | loss = 0.240, acc = 0.914
Epoch 4 | Step 60000 | loss = 0.218, acc = 0.920
Epoch 4 | Step 61000 | loss = 0.177, acc = 0.927
Epoch 4 | Step 62000 | loss = 0.185, acc = 0.923
Epoch 4 | Step 63000 | loss = 0.201, acc = 0.924
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

Validation | Epoch 4 | acc = 0.792
Saving Model ... saved_model18


In [21]:
if logToWandb:
    wandb.finish()

## Testing

In [22]:
print("Evaluating Test Set ...")

result = []
model = model.from_pretrained("saved_model{}".format(Numbering)).to(device)
model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result{}.csv".format(Numbering)
with open(result_file, 'w', encoding= 'utf-8-sig') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

Completed! Result is in result18.csv
